<a href="https://colab.research.google.com/github/YunSeoHwan/My-own-study-repository/blob/main/5-2(finish).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **검증 세트**

In [15]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

# 훈련세트 테스트세트 나누기
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
# 검증세트 나누기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
print(sub_input.shape, val_input.shape)

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

(4157, 3) (1040, 3)
0.9971133028626413
0.864423076923077


# **교차 검증**

In [16]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores) # 처음 2개 키는 모델훈련시간, 검증시간

import numpy as np
print(np.mean(scores['test_score'])) # 마지막 키 test_score의 평균 즉, 검증 폴드의 점수

{'fit_time': array([0.01048946, 0.00724792, 0.00787234, 0.00845242, 0.00712466]), 'score_time': array([0.00082397, 0.00072765, 0.00078726, 0.00077915, 0.00073409]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


# **분할기를 이용한 교차검증(위 값과 동일)**

In [17]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


# **10-폴드 교차 검증**

In [19]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# **하이퍼파라미터 튜닝**

In [20]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
    

In [21]:
# 검증점수가 가장 높은 모델
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [22]:
# 최적의 매개변수
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [23]:
# 교차검증으로 얻은 값
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [24]:
# argmax 함수
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [25]:
# 복잡한 매개변수
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [26]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [27]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


# **랜덤 서치**

In [28]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([5, 5, 0, 1, 7, 1, 8, 6, 7, 5])

In [31]:
np.unique(rgen.rvs(1000), return_counts=True) # 0~9로 이루어진 1000개의 숫자 반복 횟수 -> unique는 겹치는 값 제거

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 94, 103, 103, 100, 101,  93, 115, 104,  86, 101]))

In [34]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.42461442, 0.2250617 , 0.08352862, 0.31204712, 0.70306632,
       0.88533717, 0.3119629 , 0.65751201, 0.16574793, 0.39672621])

In [35]:
# 실전연습
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [36]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [37]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [38]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
